# A3Data - Residência Porto Digital

## Processamento de Dados em Larga Escala - Aula 02


---
### Setup

In [1]:
%%bash
# Instal Java
apt-get update && apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install PySpark
pip install -q pyspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu b

In [2]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AtividadeSQL").getOrCreate()

def q(query, n=30):
    return spark.sql(query).show(n=n, truncate=False)

### Datasets para a atividade de hoje

In [3]:
%%bash
curl -O https://raw.githubusercontent.com/neylsoncrepalde/campeonato_brasileiro_dataset/main/campeonato-brasileiro-gols.csv
curl -O https://raw.githubusercontent.com/neylsoncrepalde/campeonato_brasileiro_dataset/main/campeonato-brasileiro-full.csv
curl -O https://raw.githubusercontent.com/neylsoncrepalde/campeonato_brasileiro_dataset/main/campeonato-brasileiro-estatisticas-full.csv
curl -O https://raw.githubusercontent.com/neylsoncrepalde/campeonato_brasileiro_dataset/main/campeonato-brasileiro-cartoes.csv
curl -O https://raw.githubusercontent.com/neylsoncrepalde/campeonato_brasileiro_dataset/main/Legenda.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  270k  100  270k    0     0  1367k      0 --:--:-- --:--:-- --:--:-- 1367k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  825k  100  825k    0     0  2636k      0 --:--:-- --:--:-- --:--:-- 2628k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  600k  100  600k    0     0  2099k      0 --:--:-- --:--:-- --:--:-- 2099k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  855k  100  855k    0     0  3238k      0 --:--:-- --:--:-- --:--:-- 3238k
  % Total    % Received % Xferd  Average Speed   Tim

In [4]:
!cat Legenda.txt

Legenda - campeonato-brasileiro-full.csv
ID - ID da partida
Rodada : Rodada que aconteceu a partida
Data : Data que ocorreu a partida
Horário :  Horário que ocorreu a partida
Dia : Dia da semana que ocorreu a partida
Mandante : Clube mandante                      
Visitante : Clube Visitante
formacao_mandante: Formação do mandante
formacao_visitante: Formação do visitante
tecnico_mandante: Técnico do mandante
tecnico_visitante: Técnico do visitante
Vencedor : Clube vencedor da partida. Quando tiver "-", é um empate
Arena : Arena que ocorreu a partida                      
Mandante Placar : Gols que o clube mandante fez na partida                      
Visitante Placar : Gols que o clube visitante fez na partida                      
Estado Mandante : Estado do clube mandatorio                      
Estado Visitante : Estado do clube visitante                      
Estado Vencedor : Estado do clube vencedor. Quando tiver "-", é um empate


Legenda - campeonato-brasileiro-estatisticas-fu

In [5]:
parametros = {
    'header': True,
    'inferSchema': True,
    'sep': ','
}

schema_full = """ID int, rodada int, data string, hora string, dia string, mandante string, visitante string, formacao_mandante string, 
  formacao_visitante string, tecnico_mandante string, tecnico_visitante string, vencedor string, arena string, mandante_placar string,
  visitante_placar string, mandante_estado string, visitante_estado string, estado_vencedor string"""

cartoes = spark.read.csv("campeonato-brasileiro-cartoes.csv", **parametros)
gols = spark.read.csv("campeonato-brasileiro-gols.csv", **parametros)
estats = spark.read.csv("campeonato-brasileiro-estatisticas-full.csv", **parametros)
full = spark.read.csv("campeonato-brasileiro-full.csv", **parametros, schema=schema_full)

cartoes.createOrReplaceTempView('cartoes')
gols.createOrReplaceTempView('gols')
estats.createOrReplaceTempView('estats')
full.createOrReplaceTempView('full')

In [6]:
print('CARTOES')
cartoes.printSchema()
print('GOLS')
gols.printSchema()
print('ESTATS')
estats.printSchema()
print('FULL')
full.printSchema()

CARTOES
root
 |-- partida_id: integer (nullable = true)
 |-- rodada: integer (nullable = true)
 |-- clube: string (nullable = true)
 |-- cartao: string (nullable = true)
 |-- atleta: string (nullable = true)
 |-- num_camisa: integer (nullable = true)
 |-- posicao: string (nullable = true)
 |-- minuto: string (nullable = true)

GOLS
root
 |-- partida_id: integer (nullable = true)
 |-- rodada: integer (nullable = true)
 |-- clube: string (nullable = true)
 |-- atleta: string (nullable = true)
 |-- minuto: string (nullable = true)

ESTATS
root
 |-- partida_id: integer (nullable = true)
 |-- rodada: integer (nullable = true)
 |-- clube: string (nullable = true)
 |-- chutes: integer (nullable = true)
 |-- chutes_no_alvo: integer (nullable = true)
 |-- posse_de_bola: string (nullable = true)
 |-- passes: integer (nullable = true)
 |-- precisao_passes: string (nullable = true)
 |-- faltas: integer (nullable = true)
 |-- cartao_amarelo: integer (nullable = true)
 |-- cartao_vermelho: integer (

## Análises

In [24]:
q("""

  SELECT *
  FROM full
  WHERE dia = 'Quarta' -- esse filtro retorna uma tabela VAZIA!!
  LIMIT 20

""")

+---+------+----+----+---+--------+---------+-----------------+------------------+----------------+-----------------+--------+-----+---------------+----------------+---------------+----------------+---------------+
|ID |rodada|data|hora|dia|mandante|visitante|formacao_mandante|formacao_visitante|tecnico_mandante|tecnico_visitante|vencedor|arena|mandante_placar|visitante_placar|mandante_estado|visitante_estado|estado_vencedor|
+---+------+----+----+---+--------+---------+-----------------+------------------+----------------+-----------------+--------+-----+---------------+----------------+---------------+----------------+---------------+
+---+------+----+----+---+--------+---------+-----------------+------------------+----------------+-----------------+--------+-----+---------------+----------------+---------------+----------------+---------------+



In [20]:
q("""
  -- Quantas partidas a gente tem informação, qual foi a primeira e qual foi a última
  SELECT 
    COUNT(ID) AS contagem_de_partidas,
    MIN(data) AS data_minima,
    MAX(data) AS data_maxima
  FROM full

""")

+--------------------+-----------+-----------+
|contagem_de_partidas|data_minima|data_maxima|
+--------------------+-----------+-----------+
|7645                |2003-03-29 |2021-12-12 |
+--------------------+-----------+-----------+



In [30]:
q("""

  SELECT *
  FROM cartoes
  LIMIT 20

""")

+----------+------+-------------+-------+--------------------------+----------+----------+------+
|partida_id|rodada|clube        |cartao |atleta                    |num_camisa|posicao   |minuto|
+----------+------+-------------+-------+--------------------------+----------+----------+------+
|4607      |1     |Figueirense  |Amarelo|Thiago Heleno             |4         |Zagueiro  |44    |
|4607      |1     |Figueirense  |Amarelo|Paulo Roberto da Silva    |28        |null      |66    |
|4608      |1     |Internacional|Amarelo|Juan Silveira dos Santos  |4         |Zagueiro  |20    |
|4608      |1     |Vitoria      |Amarelo|Vinícius Santos Silva     |null      |null      |82    |
|4608      |1     |Internacional|Amarelo|Andrés D'Alessandro       |10        |Meio-campo|72    |
|4608      |1     |Vitoria      |Amarelo|Marcelo Machado dos Santos|29        |null      |86    |
|4608      |1     |Vitoria      |Amarelo|Mansur                    |null      |Zagueiro  |10    |
|4609      |1     |C

### Pergunta

Em quantas partidas o **Palmeiras** recebeu cartões amarelos quando jogava como visitante?

Nesse caso, preciso juntar as informações que estão em tabelas diferentes (informação de cartões na tabela cartões e informação de visitante na tabela full).
Para isso, vamos utilizar o comando **JOIN**

In [60]:
q("""

  SELECT
    COUNT(DISTINCT ID) as contagem
    --f.ID, f.rodada, f.mandante, f.visitante, c.clube, c.cartao
  FROM full f
  INNER JOIN cartoes c
    ON f.ID = c.partida_id
  WHERE f.visitante = 'Palmeiras' 
    AND c.cartao = 'Amarelo'
    AND c.clube = 'Palmeiras'

""")

+--------+
|contagem|
+--------+
|143     |
+--------+



#### Tipos de JOIN

- FULL (OUTER) JOIN - junta tudo (todas as linhas da tabela A com todas as linhas da tabela B)
- INNER JOIN - junta apenas os casos em comum.
- LEFT JOIN - traz todas as linhas da tabela A e as linhas que derem match da tabela B
- RIGHT JOIN - traz todas as linhas da tabela B e as linhas que derem match da tabela A

### Pergunta:

Qual é o total de impedimentos que o Atlético MG teve nas partidas em que marcou gol no segundo tempo?

In [97]:
q("""

  SELECT
    SUM(impedimentos) AS total_impedimentos
  FROM (                  -- SUB QUERY
    SELECT DISTINCT
      e.partida_id,
      e.clube,
      e.impedimentos
    FROM estats e
    INNER JOIN gols g
      ON e.partida_id = g.partida_id
    WHERE 
      g.clube = 'Atletico-MG'
      AND g.minuto >= 46
      AND e.clube = 'Atletico-MG'
      AND e.impedimentos > 0
    ORDER BY e.partida_id
  )

""")

+------------------+
|total_impedimentos|
+------------------+
|286               |
+------------------+



# Dever de casa

Responda as perguntinhas:

1. Qual é a média de passes que o Cruzeiro fez nas partidas em que foi vencedor?
2. Quantos cartões amarelos o Grêmio levou no segundo tempo quando jogava em casa (mandante)?
3. Quantos gols o Flamengo marcou fora de casa (visitante) nas partidas em que tomou mais de um cartão amarelo?
4. Qual é a média de passes que o Cruzeiro fez nas partidas em que foi vencedor e marcou PELO MENOS 1 gol no primeiro tempo? (PEGADINHAS)